# Modeling

There are three steps to creating this model:

1. **Vectorization**
2. **Train/Validation/Test Split**
3. **Modeling**: We apply a baseline CNN.

In [25]:
import numpy as np
import re
import keras
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras import layers
from keras.layers import Dense, Conv1D, Flatten, LSTM, GRU, GlobalMaxPooling1D
from keras import optimizers
from keras import models
from keras import losses
from keras import metrics
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from joblib import dump, load
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Import custom functions
from explore import *
from evaluation import *

In [3]:
# Import the dataset
df = pd.read_csv('processed_data/clean_data.csv')

In [6]:
df.isna().sum()

Tweet                       1830
Years                          0
Lemmatized                  3220
Tweets with no Stopwords    6230
Short Tweets                6893
dtype: int64

In [7]:
df = df.dropna()

In [24]:
print(df['Years'].unique())

[2018 2017 2019]


# CNN for Text Analysis

Neural networks analyze texts in a slightly different way with words as opposed to the sparse TF-IDF framework. Since this is a large dataset, a CNN maybe able to pick up intricate patterns. Preprocessing with CNNs requires it to be processed with Keras' `Embedding.()` when it comes to the modeling.

In [18]:
X_train, X_test, y_train, y_test = train_test_split(df['Tweets with no Stopwords'], 
                                                    df['Years'], test_size = 0.3, random_state = 2)

In [20]:
token = Tokenizer(num_words = 5000, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{"}~\t\n', lower = True, 
                  split = ' ', oov_token = True)
token.fit_on_texts(X_train)
X_train_seq = token.texts_to_sequences(X_train)
X_test_seq = token.texts_to_sequences(X_test)

In [28]:
# One hot encoding for labels
encoder = LabelBinarizer()
encoder.fit(y_train)
transformed = encoder.transform(y_train)
y_train_encoded = pd.DataFrame(transformed)
transformed_test = encoder.transform(y_test)
y_test_encoded = pd.DataFrame(transformed_test)

In [21]:
max_seq_length(X_train_seq)

83

In [29]:
max_length = 83

In [22]:
# Total number of words in the corpus
vocabulary_size = len(token.word_index)
vocabulary_size

374556

In [23]:
number_of_words = 5000

In [30]:
X_train_seq_pad = pad_sequences(X_train_seq, maxlen = max_length, padding = 'post')
X_test_seq_pad = pad_sequences(X_test_seq, maxlen = max_length, padding = 'post')

In [31]:
X_train_emb, X_val_emb, y_train_emb, y_val_emb = train_test_split(X_train_seq_pad, y_train_encoded, 
                                                                  test_size = 0.3, random_state = 3)

In [34]:
emb_model = models.Sequential()
emb_model.add(layers.Embedding(number_of_words, 200, input_length = max_length))
emb_model.add(layers.Conv1D(50, 3, activation = 'relu', input_shape = (200, 1)))
emb_model.add(layers.GlobalMaxPooling1D())
emb_model.add(layers.Dense(3, activation = 'softmax'))
emb_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [35]:
emb_model_history = emb_model.fit(X_train_emb, y_train_emb, epochs = 10, batch_size = 64, validation_split = 0.3)

Epoch 1/10
5582/5582 [==============================] - 234s 42ms/step - loss: 0.4772 - accuracy: 0.8313 - val_loss: 0.4219 - val_accuracy: 0.8487
Epoch 2/10
5582/5582 [==============================] - 235s 42ms/step - loss: 0.3918 - accuracy: 0.8549 - val_loss: 0.4318 - val_accuracy: 0.8429
Epoch 3/10
5582/5582 [==============================] - 233s 42ms/step - loss: 0.3493 - accuracy: 0.8673 - val_loss: 0.4351 - val_accuracy: 0.8432
Epoch 4/10
5582/5582 [==============================] - 233s 42ms/step - loss: 0.3002 - accuracy: 0.8863 - val_loss: 0.4724 - val_accuracy: 0.8313
Epoch 5/10
5582/5582 [==============================] - 233s 42ms/step - loss: 0.2548 - accuracy: 0.9045 - val_loss: 0.5157 - val_accuracy: 0.8279
Epoch 6/10
5582/5582 [==============================] - 230s 41ms/step - loss: 0.2140 - accuracy: 0.9216 - val_loss: 0.5731 - val_accuracy: 0.8266
Epoch 7/10
5582/5582 [==============================] - 234s 42ms/step - loss: 0.1803 - accuracy: 0.9344 - val_loss: 0

In [36]:
emb_model.evaluate(X_val_emb, y_val_emb)

6834/6834 [==============================] - 35s 5ms/step - loss: 0.8500 - accuracy: 0.8045


[0.8499530553817749, 0.8045334219932556]

In [38]:
print(X_test_seq)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
emb_model.evaluate(X_test_seq_pad, y_test_encoded)

1584/9763 [===>..........................] - ETA: 42s - loss: 0.8522 - accuracy: 0.8059